<a href="https://colab.research.google.com/github/edgoni/NN-QS-MBSpinSystems/blob/main/basic_selfAtt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install netket

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.5/833.5 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 4.9 MB/s eta 0:00:00


In [45]:
!git config --global user.email "eduardogonicrespo@gmail.com"
!git config --global user.name "edgoni"

!git clone https://github.com/edgoni/NN-QS-MBSpinSystems

In [46]:
import numpy as np
import netket as nk

import jax
import jax.numpy as jnp

import flax
import flax.linen as nn

In [31]:
input = np.random.rand(18) # aleatorio (18)
print(input.shape)

(18,)


In [32]:
input = input.reshape(1, -1)
print(input.shape)

(1, 18)


In [41]:
class Basic_SelfAtt(nn.Module):
  ### Parametros
  layers: int #capas
  heads: int # cabezas de attention
  dk: int #dimension matrices entrenables

  ###call
  @nn.compact
  def __call__(self, x):
    dimension = x.shape[-1]

    #definimos matrices de entrenamiento

    weight_Q = self.param(f'weight_Q', nn.initializers.xavier_uniform(), (dimension, dimension))
    weight_K = self.param(f'weight_K', nn.initializers.xavier_uniform(), (dimension, dimension))
    weight_V = self.param(f'weight_V', nn.initializers.xavier_uniform(), (dimension, dimension))

    # Bloque Self_Attention
    Q = jnp.dot(x, weight_Q)
    K = jnp.dot(x, weight_K)
    V = jnp.dot(x, weight_V)

    QK_matmul = jnp.einsum('nd,de->ne', Q, jnp.transpose(K)) /jnp.sqrt(dimension)

    #QK_matmul = jnp.matmul(Q, jnp.transpose(K)) # o einsum????
    softmax = jax.nn.softmax(QK_matmul)

    last_matmul = jnp.matmul(softmax, V)

    output = nn.LayerNorm()(last_matmul)
    output = nn.Dense(1)(output) # log amplitud final

    return output

In [50]:
class Layers_SelfAtt(nn.Module):
  ### Parametros
  layers: int #capas
  heads: int # cabezas de attention
  #dk: int #dimension matrices entrenables

  ###call
  @nn.compact
  def __call__(self, x):
    dimension = x.shape[-1]

    #definimos matrices de entrenamiento
    for i in range(self.layers):

      #-----------------------------------------------------------------------------------------------#

      weight_Q = self.param(f'weight_Q_head{i}', nn.initializers.xavier_uniform(), (dimension, dimension))
      weight_K = self.param(f'weight_K_head{i}', nn.initializers.xavier_uniform(), (dimension, dimension))
      weight_V = self.param(f'weight_V_head{i}', nn.initializers.xavier_uniform(), (dimension, dimension))

      # Bloque Self_Attention
      Q = jnp.dot(x, weight_Q)
      K = jnp.dot(x, weight_K)
      V = jnp.dot(x, weight_V)

      QK_matmul = jnp.einsum('id,jd->ij', Q, K) / jnp.sqrt(dimension)

      #QK_matmul = jnp.matmul(Q, jnp.transpose(K)) # o einsum????
      softmax = jax.nn.softmax(QK_matmul)

      att_output = jnp.matmul(softmax, V)

      #-----------------------------------------------------------------------------------------------#
      #ESTAMOS EN UN BUCLE IMPORTANTE PONER NOMBRES
      ##Residuo2##
      x = x + att_output
      x = nn.LayerNorm(name=f'ln1_{i}')(x)

      ##MLP##
      mlp = nn.Dense(dimension * 4, name=f'mlp_up_{i}')(x)
      mlp = nn.gelu(mlp)
      mlp = nn.Dense(dimension, name=f'mlp_down_{i}')(mlp)

      ##Residuo2##
      x = x + mlp
      x = nn.LayerNorm(name=f'ln2_{i}')(x)


    output = nn.Dense(1)(x) # log amplitud final

    return output

In [56]:
SelfAtt_model = Basic_SelfAtt(layers=1, heads=4, dk=1)

In [57]:
key = jax.random.PRNGKey(0)
params = SelfAtt_model.init(key, input)
output = SelfAtt_model.apply(params, input)
print(output)

[[-0.03119992]]


In [49]:
!git add

fatal: not a git repository (or any of the parent directories): .git


In [47]:
!git commit -m "mensaje"

fatal: not a git repository (or any of the parent directories): .git
